In [1]:
from load_dataset import load_data
import numpy as np
import torch
from torch.utils.data import TensorDataset,DataLoader
from UNet import UNet
from conbine_loss import CombinedLoss
import torch.optim as optim
import torch.nn as nn
path_test = "../input0/test_npz_seg_512x512_s1"
path_train ='../input0/train_npz_seg_512x512_s1'
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [2]:
torch.cuda.is_available()

True

In [3]:
np_test,np_test_mask = load_data(path_test)
np_train,np_train_mask = load_data(path_train)
# print(np_test.shape,np_test_mask.shape)
# print(np_train.shape,np_train_mask.shape)
# 归一化
test_data = (np_test/255).astype(np.float32)
test_mask = np_test_mask
train_data = (np_train/255).astype(np.float32)
train_mask = np_train_mask
train_data =torch.from_numpy(train_data)
train_data = train_data.permute(0, 3, 1, 2)
test_data = torch.from_numpy(test_data)
test_data = test_data.permute(0, 3, 1, 2)
## print(train_data.shape)
## print(train_mask.shape)


In [4]:
test_set = TensorDataset(test_data, torch.from_numpy(test_mask))
train_set = TensorDataset(train_data, torch.from_numpy(train_mask))

net = UNet(n_channels=3,n_classes=26).to(device)
criterion = nn.CrossEntropyLoss()#定义损失函数
optimizer = optim.Adam(net.parameters(), lr=0.010)  # 定义优化器
trainLoader = DataLoader(train_set, batch_size=2,
                         shuffle=True, num_workers=0)
testLoader = DataLoader(test_set, batch_size=1,
                         shuffle=False, num_workers=0)


In [ ]:
import time
start = time.time()
for epoch in range(100):
    running_loss = 0.0
    for i, data in enumerate(trainLoader, 0):
        inputs, labels = data  # 获取数据
        inputs = inputs.to(device)
        labels = labels.to(device)
        # print(inputs.shape)
        optimizer.zero_grad()  # 清空梯度缓存
        outputs = net(inputs)
        # print(outputs.shape)
        loss = criterion(outputs, labels.long())
        loss.backward()  # 反向传播
        optimizer.step()  # 调整模型
        running_loss += loss.item()
        if i % 10 == 9:
            # 每 20 次迭代打印一次信息
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss))
            running_loss = 0.0
print('Finish Traning! Total cost time: ', time.time()-start)



[1,    10] loss: 18.077
[1,    20] loss: 3.870
[1,    30] loss: 3.287
[1,    40] loss: 3.339
[1,    50] loss: 3.376
[1,    60] loss: 3.067
[1,    70] loss: 3.186
[1,    80] loss: 3.131
[1,    90] loss: 3.118
[1,   100] loss: 3.425
[1,   110] loss: 2.892
[1,   120] loss: 2.966
[1,   130] loss: 3.341
[1,   140] loss: 3.029
[1,   150] loss: 3.274
[1,   160] loss: 3.125
[1,   170] loss: 3.069
[1,   180] loss: 3.042
[1,   190] loss: 3.063
[1,   200] loss: 2.973
[1,   210] loss: 2.965
[1,   220] loss: 3.174
[1,   230] loss: 3.220
[1,   240] loss: 2.842
[1,   250] loss: 3.119
[1,   260] loss: 3.189
[1,   270] loss: 2.881
[2,    10] loss: 2.973
[2,    20] loss: 2.923
[2,    30] loss: 3.091
[2,    40] loss: 2.953
[2,    50] loss: 3.030
[2,    60] loss: 2.697
[2,    70] loss: 3.011
[2,    80] loss: 3.028
[2,    90] loss: 2.854
[2,   100] loss: 2.918
[2,   110] loss: 3.227
[2,   120] loss: 2.892
[2,   130] loss: 2.992
[2,   140] loss: 3.170
[2,   150] loss: 2.976
[2,   160] loss: 2.737
[2,   170]

In [ ]:
torch.save(net, 'net.pkl ')